In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from scipy import sparse

# Generate some indices
Even the sparse matrices won't fit in memory. So we will have to loop through them when making predictions or sampling random items.

In [2]:
#count number of items:
indptr = [0]

for chunkID in range(12):
    scores = np.load(f'../processed_data/D4_all{chunkID}.npy')
    indptr.append(indptr[-1] + scores.shape[0])


In [3]:
scores = np.concatenate([np.load(f'../processed_data/D4_all{i}.npy') for i in range(12)])

# functions to handle the slabs

In [4]:
def extractFPs(chunkID, indptr, isTrain):
    fp = sparse.load_npz(f'../processed_data/D4_all{chunkID}.npz')
    mask = isTrain[indptr[chunkID]:indptr[chunkID+1]]
    return fp[mask]

def buildTrain(indptr, isTrain, verbose=0):
    if verbose:
        print('building training matrix')
    fps = sparse.vstack([extractFPs(i, indptr, isTrain) for i in range(12)])
    return fps

def chunkPredictProba(model, indptr, isTrain, verbose=0):
    if verbose:
        print('predicting probabilities')
    probas = []
    for chunkID in range(12):
        fps = extractFPs(chunkID, indptr, ~isTrain)
        proba = model.predict_proba(fps)[:,1]
        probas.append(proba)
    return np.concatenate(probas)

# Train and RF regressor and Logistic Regression models

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000, C=0.1)

In [6]:
#topK = (scores.argsort().argsort() < (scores.shape[0]*0.0005)) #0.05th percentile.

topK = (scores.argsort().argsort() < 50_000) #~0.05th percentile.
#tot = topK.sum() 

In [10]:
tot = topK.sum()

# With Altair, using three repeats:

In [11]:

trainingSetSizes=[400_000, 200_000, 100_000]
#for percentile in [0.05, 0.1, 0.25, 0.5, 0.75, 1 ]:
for percentile in [0.1, 0.125, 0.15]:
    
    df = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
    count=0
    
    for i in range(3):
        idx = np.arange(scores.shape[0])
        np.random.shuffle(idx)

        for size in trainingSetSizes:
            #split indices into train and test:
            train = idx[:size].copy()
            test = idx[size:].copy()
            train.sort()
            test.sort()
    
            #generate a 'is a training instance' mask. 
            isTrain = np.zeros(scores.shape[0]).astype(bool)
            isTrain[train]=True
    
            #topK molecules already found in the training set:
            numFound = topK[train].sum()
        
            df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
            count+=1
            print(count, numFound)
            
            #estimate the cutoff once, from the initial random sample:
            cutoff = np.percentile(scores[train], percentile)
    
            for i in range(5):
                
                #fit model:
                model.fit(buildTrain(indptr, isTrain, 1), scores[isTrain]<cutoff)
    
                #predict (slowest step):
                proba = chunkPredictProba(model, indptr, isTrain, 1)
    
                #rank the probabilities
                proba_sorted = (-proba).argsort()
        
                #rank the unseen instances:
                test = test[proba_sorted]

                #now append the next N instances from the rank ordered unseen instances onto the training set:
                train = np.concatenate([train, test[:size]])
        
                #update the isTrain mask:
                isTrain[train]=True
        
                #now remove those training instances from the test set:
                test = test[size:]

                #keep the train and test idx arrays sorted so they agree with the chunked* methods:
                test.sort()
                train.sort()
        
                #topK molecules already found in the training set:
                numFound = topK[train].sum()
            
                df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
                count+=1
                print(count, numFound)
                
    df.to_csv('../processed_data/D4_reconstruction_'+str(percentile)+'.csv')

1 186
building training matrix
predicting probabilities
2 12591
building training matrix
predicting probabilities
3 21237
building training matrix
predicting probabilities
4 26709
building training matrix
predicting probabilities
5 30253
building training matrix
predicting probabilities
6 32849
7 95
building training matrix
predicting probabilities
8 6707
building training matrix
predicting probabilities
9 13798
building training matrix
predicting probabilities
10 19435
building training matrix
predicting probabilities
11 23242
building training matrix
predicting probabilities
12 25897
13 50
building training matrix
predicting probabilities
14 3664
building training matrix
predicting probabilities
15 8264
building training matrix
predicting probabilities
16 12740
building training matrix
predicting probabilities
17 16393
building training matrix
predicting probabilities
18 19177
19 151
building training matrix
predicting probabilities
20 11136
building training matrix
predicting probab

# Plot

In [17]:
import altair as alt
import pandas as pd
import numpy as np

In [42]:
#df = pd.read_csv('../processed_data/ampc_reconstruction_0.1.csv')
df = pd.read_csv('../processed_data/ampc_reconstruction_0.125.csv')
#df = pd.read_csv('../processed_data/ampc_reconstruction_0.15.csv')
#df = pd.read_csv('../processed_data/ampc_reconstruction_0.175.csv')


In [44]:
prev_results = [['RF (Coley)', 400_000, 71.4, 2.1], ['NN (Coley)', 400_000, 74.7, 1.4],
                ['MPN (Coley)',400_000, 87.9, 2.3],
    ['RF (Coley)', 200_000, 45.5, 1.8],
['NN (Coley)', 200_000, 52.8, 0.5],
['MPN (Coley)', 200_000, 67.1, 2.1],
['RF (Coley)', 100_000, 24.0, 2.2],
['NN (Coley)', 100_000 , 33.3,0.3],
['MPN (Coley)', 100_000, 52.0, 0.5]]


In [45]:


coley = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
count = 0 
for res in prev_results:
    
    desired_std_dev = res[3]
    samples = np.array([-1,0,1]).astype(float)
    samples *= (desired_std_dev/np.std(samples))
    for s in samples:
        coley.loc[count]= [res[0], res[1], res[1]*6, (s+res[2])/100]
        count+=1

In [46]:
concat = pd.concat([df, coley])

In [47]:
error_bars = alt.Chart(concat).mark_errorbar(extent='ci').encode(
  x=alt.X('N ligands explored:Q',title='Number of ligands sampled'),
  y=alt.Y('% top-k found:Q', title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

points = alt.Chart(concat).mark_point(filled=True, color='black').encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

line = alt.Chart(concat).mark_line(color='black',size=1,opacity=0.5).encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

ch = (error_bars+points+line).properties(height=400,width=200).facet(
    column=alt.Column('Training size:N',sort=alt.Sort([0.004, 0.002, 0.001])),
).resolve_scale(x='independent')
ch


alt.FacetChart(...)

In [106]:
#df.to_csv('../processed_data/ampc_reconstruction.csv')

In [107]:
chs = []
for frac in [400000, 200000, 100000]:
    
    df_ = concat[concat['Training size']==frac].replace('morgan_feat', 'Morgan pharm. & Log.reg. (ours)')
    error_bars = alt.Chart(df_).mark_errorbar(extent='ci').encode(
          x=alt.X('N ligands explored:Q', 
                  title='Number of ligands sampled',
                  scale=alt.Scale(domain=[0,max(df_['N ligands explored'])+10000])),
          y=alt.Y('% top-k found:Q',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    points = alt.Chart(df_).mark_point(filled=True, color='black').encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    line = alt.Chart(df_).mark_line(color='black',size=1,opacity=0.5).encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )
    ch = (error_bars+points+line).properties(width=200)
    ch.title = str(frac / (100*1e6)*100)
    chs.append(ch)


In [110]:
chs[0]
sup = chs[0] |  chs[1] | chs[2]
sup.save('../figures/ampC_reconstruction.html') #using 0.05

In [109]:
chs[0]+chs[1]+chs[2]

alt.LayerChart(...)

In [289]:
sup.save('../figures/ampC_reconstruction.html')